In [25]:
from bs4 import BeautifulSoup
import six.moves.urllib as urllib2
import re
import requests
import csv
import io
import time


# Get all of the Urls

In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import grequests

In [27]:
data = pd.read_csv("Sample_Program_Service_Data.csv", encoding='latin')

FileNotFoundError: [Errno 2] No such file or directory: 'Sample_Program_Service_Data.csv'

In [ ]:
data.columns

In [ ]:
data.head(5)[["WEBSITE", "NAME", "ADDRESS", "ZIP", "URL", "FISYR"]]

In [ ]:
float(len(data[pd.isnull(data.WEBSITE)])) / len(data)

In [ ]:
OkayWebsites = data[pd.isnull(data.WEBSITE) == False]
OkayWebsites.head(5)[["WEBSITE", "NAME", "ADDRESS", "ZIP", "URL", "FISYR"]]

In [ ]:
def exception_handler(request, exception):
    None
    #print "Request failed: " + str(request.url) + "; Exception: " + str(exception)

In [ ]:
def ProcessText(text):
    soup = BeautifulSoup(text);
    return list(map(lambda link: link.get('href'), soup.find_all('a', attrs={'href': re.compile('twitter.com')})))

def ProcessRequest(requestsPerWebsite):
    #print(requestsPerWebsite);
    if (requestsPerWebsite[0] is not None and requestsPerWebsite[0].ok):
        return ProcessText(requestsPerWebsite[0].text);
    if (len(requestsPerWebsite) == 2):
        if (requestsPerWebsite[1] is not None and requestsPerWebsite[1].ok):
            return ProcessText(requestsPerWebsite[1].text);
    return [];

In [ ]:
hdr = {'User-Agent': 'Mozilla/5.0'}
ishttp = re.compile('^http(s*):\/\/.*', flags=re.IGNORECASE)
def QueryWebsite(website):
    if (ishttp.match(website) is None):
        requests = ["http://" + website, "https://" + website]
    else:
        requests = [ website ]
    return list(grequests.get(u, timeout=4) for u in requests)

def ProcessWebsiteBatch(websiteBatch):
    unsentRequests = list(map(lambda x: QueryWebsite(x), websiteBatch))
    unsentRequestLengths = list(map(lambda x: len(x), unsentRequests))
    flattenedRequests = []
    for unsent in unsentRequests:
        flattenedRequests = flattenedRequests + unsent
    
    print("Flattened Requests: " + str(len(flattenedRequests)))
    
    reqs = grequests.map(flattenedRequests, exception_handler=exception_handler)
    idx = 0
    unFlattenedResponses = []
    for l in unsentRequestLengths:
        unFlattenedResponses = unFlattenedResponses + [reqs[idx : idx + l]]
        idx = idx + l

    # print("Unflattened Requests: " + str(len(unFlattenedResponses)))
    
    return map(lambda x: ProcessRequest(x), unFlattenedResponses)

In [ ]:
def BatchColumnRun(df, batchSize, csvwriter, csvfile, num_rows=None):
    num_rows = num_rows or len(df)
    finalNum = int(num_rows / batchSize)
    if (num_rows % batchSize != 0):
        finalNum = finalNum + 1
        
    for i in range(0, finalNum):
        division = df[i*batchSize:min((i+1)*batchSize, num_rows)]
        twitterResults = ProcessWebsiteBatch(division["WEBSITE"])
        for idx, twitterResult in enumerate(twitterResults):
            csv_writer.writerow([division["EIN"].values[idx], division["FISYR"].values[idx], twitterResult])
            csvfile.flush()
        print("Written: " + str((i + 1) * batchSize))


In [ ]:
with io.open('TwitterHandleOutput.csv', 'w') as csvfile:
    csv_writer = csv.writer(csvfile, delimiter='\t')
    # Write the header
    csv_writer.writerow(["EIN", "FISYR", "TWITTERHANDLE"])
    
    BatchColumnRun(OkayWebsites, 25, csv_writer, csvfile)
    csvfile.close()